In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import utils.data as du
import utils.plots as pu
import utils.training as tu
import utils.models as models

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch
import torch.utils.data as tdu
import mlflow

In [ ]:
scan_dataset = du.PosDataDatset('../../Data/robo_steer/raw/', 200)
train_size = int(0.8 * len(scan_dataset))
test_size = len(scan_dataset) - train_size
train_dataset, test_dataset = tdu.random_split(scan_dataset, [train_size, test_size])

In [ ]:
batch_size = 16
train_loader = tdu.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = tdu.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
model = models.MLP(200, 512, 1)

In [ ]:
mlflow.set_experiment('Manual')
mlflow.start_run()
mlflow_step = 0

In [ ]:
optimizer = optim.AdamW(model.parameters(), lr=0.0001)
#criterion = nn.MSELoss()
criterion = nn.L1Loss()
num_epochs = 150

In [ ]:
trainer = tu.Trainer(
    train_loader,
    test_loader,
    model, 
    optimizer,
    criterion
)

In [ ]:
for epoch in range(num_epochs):
    trainer.train(1)
    print(f'Finished Epoch {epoch}')

In [ ]:
fig = pu.graph_sample(model, scan_dataset, test_dataset)

In [ ]:
fig = pu.plot_diff_hist(model, scan_dataset, test_dataset)